# 使用計算服務

當您以 Azure Machine Learning 作業身分執行腳本時，您必須定義作業執行的執行內容。 其中一個金鑰組態是執行腳本的計算目標。 這可能是本機工作站 (在此情況下，計算實例) ，或遠端計算目標，例如隨選布建的 Azure Machine Learning 受控計算叢集。

在此筆記本中，您將建立計算叢集，並探索作業的計算目標。

## 開始之前

您將需要最新版的  **azureml-ai-ml** 套件，才能在此筆記本中執行程式碼。 執行下列資料格以確認已安裝它。

> **注意**：
> 如果未安裝 **azure-ai-ml** 套件，請執行 `pip install azure-ai-ml` 以安裝它。

In [ ]:
## 連線到您的工作區

安裝必要的 SDK 套件之後，現在您已準備好連線到您的工作區。

若要連線到工作區，我們需要識別碼參數 - 訂用帳戶識別碼、資源組名和工作區名稱。 資源組名和工作區名稱已為您填入。 您只需要訂用帳戶識別碼才能完成命令。

若要尋找必要的參數，請按一下 Studio 右上方的訂用帳戶和工作區名稱。 窗格會在右側開啟。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 複製訂用帳戶識別碼，並將 **YOUR-SUBSCRIPTION-ID** 取代為您複製的值。 </p>

## 建立計算叢集

在許多情況下，您的本機計算資源可能不足以處理需要處理大量資料的複雜或長時間執行的實驗;而且您可能想要利用動態建立和使用雲端中的計算資源的能力。 Azure Machine Learning 支援一系列計算目標，您可以在工作步調中定義並用來執行作業;只有在使用資源時，才支付資源的費用。

您可以使用 Python SDK 或 Azure CLI，在[Azure Machine Learning 工作室](https://ml.azure.com)中建立計算叢集。 下列程式碼資料格會檢查您的工作區是否有計算叢集名稱 `aml-cluster` 存在，如果不存在，則會建立它。

In [ ]:
建立計算叢集之後，您只能變更下列專案的組態：

- `min_instances`：節點數目下限
- `max_instances`：節點數目上限
- `idle_time_before_scale_down`：相應減少之前的閒置時間

目前，您的計算叢集 `aml-cluster` 只能調整最多一個節點。 讓我們將它變更為兩個，以允許平行計算。

In [ ]:
當計算叢集更新時，您可以列印其屬性來驗證其設定。

## 建立腳本以定型模型

若要定型模型，您必須先在**src**資料夾中建立**diabetes_training.py**腳本。 腳本會使用與定型資料相同資料夾中 ** 的diabetes.csv** 檔案。

In [ ]:
## 在計算叢集上執行作業

現在，您已準備好在您所建立的計算叢集上執行作業。

> **注意**：
> 作業需要一些時間才能啟動，因為計算叢集需要從零調整為一個節點。 一旦計算叢集準備就緒，腳本就會執行。 當作業完成時，計算叢集會相應減少為零個節點。 您可以在 [ **計算** ] 頁面中檢閱計算叢集的狀態。

After you've created a compute cluster, you can only change the configuration for:

- `min_instances`: Minimum number of nodes
- `max_instances`: Maximum number of nodes
- `idle_time_before_scale_down`: Idle time before scale down

Currently, your compute cluster `aml-cluster` can only scale do a maximum of one node. Let's change that to two, to allow for parallel compute.

In [ ]:
from azure.ai.ml.entities import AmlCompute

cluster_scale = AmlCompute(
    name="aml-cluster",
    max_instances=2,
)
ml_client.begin_create_or_update(cluster_scale)

When the compute cluster is updated, you can verify its configuration by printing its attributes.

In [ ]:
cpu_cluster = ml_client.compute.get("aml-cluster")

print (
        f"AMLCompute with name {cpu_cluster.name} has a maximum of {cpu_cluster.max_instances} nodes"
    )

## Create a script to train a model

To train a model, you'll first create the **diabetes_training.py** script in the **src** folder. The script uses the **diabetes.csv** file in the same folder as the training data.

In [ ]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

## Run a job on a compute cluster

Now, you're ready to run the job on the compute cluster you created.

> **Note**:
> The job will take some time to start as the compute cluster will need to scale from zero to one node. Once the compute cluster is ready, the script will be run. When the job has finished, the compute cluster will scale back down to zero nodes. You can review the compute cluster's status in the **Compute** page.

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-cluster",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)